<a href="https://colab.research.google.com/github/JaonHax/Experiments/blob/master/Training_GPT_2_using_TPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
if True:
  import os, shutil, nltk
  from google.colab import drive
  drive.mount('/content/drive')
  try:
    shutil.copytree('/content/drive/My Drive/gpt-2', '/content/gpt-2')
  except FileExistsError:
    print("GPT-2 folder already copied from Drive.")
  except FileNotFoundError:
    print("You don't have a folder of the required GPT-2 library in your drive! Grabbing it for you...")
    !git clone https://github.com/shawwn/gpt-2 -b tpu /content/gpt-2
    shutil.copytree('/content/gpt-2','/content/drive/My Drive/gpt-2')

  def copy_checkpoint_folder_from_drive(folder):
    try:
      shutil.copytree(os.path.join('/content/drive/My Drive', chkpt_to_grab), os.path.join('/content/gpt-2/checkpoint', chkpt_to_grab))
    
    except FileNotFoundError:
      print("Checkpoint not found! Try again.")
      return False
    
    except Exception as e:
      print(e)
      return False
    return True
  
  %cd /content/gpt-2
  grab_chkpt = str(input("Grab a GPT-2 checkpoint? (y/n)\n> "))

  if grab_chkpt.lower() == 'y':
    while True:
      chkpt_to_grab = input("Input checkpoint run name.\n> ")

      if copy_checkpoint_folder_from_drive(chkpt_to_grab):
        break
    
  !pip install -q regex toposort magic-wormhole gpt-2-simple
  import tensorflow as tf
  import gpt_2_simple as gpt2
  import numpy as np
  from gpt_2_simple.src import model, sample, encoder, memory_saving_gradients
  from gpt_2_simple.src.load_dataset import load_dataset, Sampler
  from gpt_2_simple.src.accumulate import AccumulatingOptimizer
  from gpt_2_simple.gpt_2 import is_mounted

  def generate(sess,
               run_name='run1',
               checkpoint_dir='checkpoint',
                model_name=None,
               model_dir='models',
               sample_dir='samples',
               return_as_list=False,
               truncate=None,
               destination_path=None,
               sample_delim='=' * 20 + '\n',
               prefix=None,
               seed=None,
               nsamples=1,
               batch_size=1,
               length=1023,
               temperature=0.7,
               top_k=0,
               top_p=0.0,
               include_prefix=True):
      """Generates text from a model loaded into memory.
  
      Adapted from https://github.com/openai/gpt-2/blob/master/src/interactive_conditional_samples.py
      """
  
      if batch_size is None:
          batch_size = 1
      assert nsamples % batch_size == 0
  
      if nsamples == 1:
          sample_delim = ''
  
      if prefix == '':
          prefix = None
  
      if model_name:
          checkpoint_path = os.path.join(model_dir, model_name)
      else:
          checkpoint_path = os.path.join(checkpoint_dir, run_name)
  
      enc = encoder.get_encoder(checkpoint_path)
      hparams = model.default_hparams()
      with open(os.path.join(checkpoint_path, 'hparams.json')) as f:
          hparams.override_from_dict(json.load(f))
  
      if prefix:
          context = tf.compat.v1.placeholder(tf.int32, [batch_size, None])
          context_tokens = enc.encode(prefix)
  
      np.random.seed(seed)
      tf.compat.v1.set_random_seed(seed)
  
      output = sample.sample_sequence(
          hparams=hparams,
          length=min(length, 1023 - (len(context_tokens) if prefix else 0)),
          start_token=enc.encoder['<|endoftext|>'] if not prefix else None,
          context=context if prefix else None,
          batch_size=batch_size,
          temperature=temperature, top_k=top_k, top_p=top_p
      )[:, 1:]
  
      if destination_path:
          f = open(destination_path, 'w')
      generated = 0
      gen_texts = []
      while generated < nsamples:
          if not prefix:
              out = sess.run(output)
          else:
              out = sess.run(output, feed_dict={
                      context: batch_size * [context_tokens]
                  })
          for i in range(batch_size):
              generated += 1
              gen_text = enc.decode(out[i])
              if prefix:
                  gen_text = enc.decode(context_tokens[:1]) + gen_text
              if truncate:
                  truncate_esc = re.escape(truncate)
                  if prefix and not include_prefix:
                      prefix_esc = re.escape(prefix)
                      pattern = '(?:{})(.*?)(?:{})'.format(prefix_esc,
                                                          truncate_esc)
                  else:
                      pattern = '(.*?)(?:{})'.format(truncate_esc)
  
                  trunc_text = re.search(pattern, gen_text, re.S)
                  if trunc_text:
                      gen_text = trunc_text.group(1)
              gen_text = gen_text.lstrip('\n')
              if destination_path:
                  f.write("{}\n{}".format(gen_text, sample_delim))
              if not return_as_list and not destination_path:
                  print("{}\n{}".format(gen_text, sample_delim), end='')
              gen_texts.append(gen_text)
  
      if destination_path:
          f.close()
  
      if return_as_list:
          return gen_texts
  
  def generate_to_file(sess,
                       run_name='run1',
                       checkpoint_dir='checkpoint',
                       model_name=None,
                       model_dir='models',
                       truncate=None,
                       destination_path='gpt_2_gen_texts.txt',
                       sample_delim='=' * 20 + '\n',
                       prefix=None,
                       seed=None,
                       nsamples=1,
                       batch_size=1,
                       length=1023,
                       temperature=0.7,
                       top_k=0,
                       top_p=0.0,
                       include_prefix=True):
      """Generates the texts to a file.
  
      sample_delim separates texts: set to '' if each text is a small document.
  
      Adapted from https://github.com/minimaxir/textgenrnn/blob/master/textgenrnn/textgenrnn.py
      """
  
      generate(sess=sess,
              run_name=run_name,
              checkpoint_dir=checkpoint_dir,
              model_name=model_name,
              model_dir=model_dir,
              return_as_list=False,
              truncate=truncate,
              destination_path=destination_path,
              sample_delim=sample_delim,
              prefix=prefix,
              seed=seed,
              nsamples=nsamples,
              batch_size=batch_size,
              length=length,
              temperature=temperature,
              top_k=top_k,
              top_p=top_p,
              include_prefix=include_prefix)
  
  def start_tf_sess(threads=-1, server=None):
      """
      Returns a tf.Session w/ config
      """
      config = tf.compat.v1.ConfigProto()
      config.gpu_options.allow_growth = True
      config.graph_options.rewrite_options.layout_optimizer = rewriter_config_pb2.RewriterConfig.OFF
      if threads > 0:
          config.intra_op_parallelism_threads = threads
          config.inter_op_parallelism_threads = threads
  
      if server is not None:
          return tf.compat.v1.Session(target=server.target, config=config)
      
      return tf.compat.v1.Session(config=config)
  
  def load_gpt2(sess,
                run_name="run1",
                checkpoint_dir="checkpoint",
                model_name=None,
                model_dir='models',
                multi_gpu=False):
      """Loads the model checkpoint or existing model into a TensorFlow session
      for repeated predictions.
      """
  
      if model_name:
          checkpoint_path = os.path.join(model_dir, model_name)
      else:
          checkpoint_path = os.path.join(checkpoint_dir, run_name)
  
      hparams = model.default_hparams()
      with open(os.path.join(checkpoint_path, 'hparams.json')) as f:
          hparams.override_from_dict(json.load(f))
  
      context = tf.compat.v1.placeholder(tf.int32, [1, None])
  
      gpus = []
      if multi_gpu:
          gpus = get_available_gpus()
  
      output = model.model(hparams=hparams, X=context, gpus=gpus)
  
      ckpt = tf.train.latest_checkpoint(checkpoint_path)
      saver = tf.compat.v1.train.Saver(allow_empty=True,
              var_list=[v for v in tf.trainable_variables() if 'model' in v.name],
              max_to_keep=5,
              keep_checkpoint_every_n_hours=2)
      sess.run(tf.compat.v1.global_variables_initializer())
  
      if model_name:
          print('Loading pretrained model', ckpt)
      else:
          print('Loading checkpoint', ckpt)
      saver.restore(sess, ckpt)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GPT-2 folder already copied from Drive.
Grab a GPT-2 checkpoint? (y/n)
> y
Input checkpoint run name.
> skip-titler-v3


FileNotFoundError: ignored

In [ ]:
!python3 download_model.py 1558M

Fetching checkpoint: 1.00kit [00:00, 817kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 49.7Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 756kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 6.23Git [02:43, 38.1Mit/s]                                 
Fetching model.ckpt.index: 21.0kit [00:00, 11.9Mit/s]                                               
Fetching model.ckpt.meta: 1.84Mit [00:00, 57.0Mit/s]                                                
Fetching vocab.bpe: 457kit [00:00, 46.3Mit/s]                                                       


In [ ]:
!python3 train.py --model_name 1558M --dataset /content/training-data.txt --batch_size 4 --save_every 50 --sample_every 10 --sample_num 1 --sample_length 200 --save_on_ctrlc --run_name skip-titler-v3 --restore_from latest --learning_rate 0.0001 --storage_bucket ""
 
with open('/content/gpt-2/checkpoint/skip-titler-v3/counter', 'r') as counter:
  for root, dirs, files in os.walk('/content/gpt-2/checkpoint/skip-titler-v3'):
    for i in range(int(counter.read())):
      for datafile in files:
        if (f"model-{i}-" in datafile):
          os.remove(os.path.join(root, datafile))

shutil.copytree('/content/gpt-2/checkpoint/skip-titler-v3', '/content/drive/My Drive/skip-titler-v3')


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


grpc://10.5.24.2:8470


Using TPU grpc://10.5.24.2:8470





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.

This model is using 1557611200 parameters (1485.45M)






Loading checkpoint gs://sgappa-multi/gpt-2/models/1558M/model.ckpt
2020-07-20 10:02:03.255891: W tensorflow/core/platform/cloud/google_auth_provider.cc:178] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could

In [ ]:
if os.path.exists('/content/checkpoint_skip-titler-v3.7z'):
  os.remove('/content/checkpoint_skip-titler-v3.7z')
 
!7z a -mx -myx -mmt4 -slp -r /content/checkpoint_skip-titler-v3.7z /content/gpt-2/checkpoint/skip-titler-v3


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive:
  0M Scan  /content/gpt-2/checkpoint/                                     1 folder, 198 files, 6230556284 bytes (5942 MiB)

Creating archive: /content/checkpoint_skip-titler-v3.7z

Items to compress: 199

  0%      0% 4 + skip-titler-v3/model-84-1.npy                                        0% 5 + skip-titler-v3/model-84-10.npy                                         0% 6 + skip-titler-v3/model-84-100.npy                                          0% 7 + skip-title

In [ ]:
shutil.copy2('/content/drive/My Drive/checkpoint_skip-titler-v3.7z', '/content/checkpoint_skip-titler-v3.7z')
!7z e -mx -myx -mmt4 -slp /content/checkpoint_skip-titler-v3.7z /content/gpt-2/checkpoint/skip-titler-v3


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 5682872060 bytes (5420 MiB)

Extracting archive: /content/checkpoint_skip-titler-v3.7z
--
Path = /content/checkpoint_skip-titler-v3.7z
Type = 7z
Physical Size = 5682872060
Headers Size = 1811
Method = LZMA2:26
Solid = +
Blocks = 2


No files to process
Everything is Ok

Files: 0
Size:       0
Compressed: 5682872060


In [ ]:
!7z -h


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Usage: 7z <command> [<switches>...] <archive_name> [<file_names>...]
       [<@listfiles...>]

<Commands>
  a : Add files to archive
  b : Benchmark
  d : Delete files from archive
  e : Extract files from archive (without using directory names)
  h : Calculate hash values for files
  i : Show information about supported formats
  l : List contents of archive
  rn : Rename files in archive
  t : Test integrity of archive
  u : Update files to archive
  x : eXtract files with full paths

<Switches>
  -- : Stop switches parsing
  -ai[r[-|0]]{@listfile|!wildcard} : Include archives
  -ax[r[-|0]]{@listfile|!wildcard} : eXclude archives
  -ao{a|s|t|u} : set Overwrite mode
  -an : disable archive_name field
  -bb[0-3] : set output log level
  -bd : disable progress indicator
  -bs{o|e|p}{0|1|2} : 

In [ ]:
!git pull

Already up to date.
